# 语言模型

语言模型（language model）是自然语言处理的重要技术。自然语言处理中最常见的数据是文本数据。我们可以把一段自然语言文本看作一段离散的时间序列。假设一段长度为$T$的文本中的词依次为$w_1, w_2, \ldots, w_T$，那么在离散的时间序列中，$w_t$（$1 \leq t \leq T$）可看作在时间步（time step）$t$的输出或标签。给定一个长度为$T$的词的序列$w_1, w_2, \ldots, w_T$，语言模型将计算该序列的概率：

$$P(w_1, w_2, \ldots, w_T).$$

**每一个词出现的概率，与前面的词已经出现过，再出现这个词的概率。**

##  语言模型的计算

既然语言模型很有用，那该如何计算它呢？假设序列$w_1, w_2, \ldots, w_T$中的每个词是依次生成的，我们有

$$P(w_1, w_2, \ldots, w_T) = \prod_{t=1}^T P(w_t \mid w_1, \ldots, w_{t-1}).$$

例如，一段含有4个词的文本序列的概率

$$P(w_1, w_2, w_3, w_4) =  P(w_1) P(w_2 \mid w_1) P(w_3 \mid w_1, w_2) P(w_4 \mid w_1, w_2, w_3).$$

我们需要计算词的概率，以及一个词在给定前几个词的情况下的**条件概率，即语言模型参数**。

**例如，$P(w_1)$可以计算为$w_1$在训练数据集中的词频** （词出现的次数）**与训练数据集的总词数之比。**

##  $n$元语法

当序列长度增加时，计算和存储多个词共同出现的概率的复杂度会呈指数级增加。$n$元语法通过马尔可夫假设（虽然并不一定成立）简化了语言模型的计算。这里的马尔可夫假设是指一个词的出现只与前面$n$个词相关，即$n$阶马尔可夫链（Markov chain of order $n$）。如果$n=1$，那么有$P(w_3 \mid w_1, w_2) = P(w_3 \mid w_2)$。如果基于$n-1$阶马尔可夫链，我们可以将语言模型改写为

$$P(w_1, w_2, \ldots, w_T) \approx \prod_{t=1}^T P(w_t \mid w_{t-(n-1)}, \ldots, w_{t-1}) .$$

以上也叫$n$元语法（$n$-grams）。它是基于$n - 1$阶马尔可夫链的概率语言模型。当$n$分别为1、2和3时，我们将其分别称作一元语法（unigram）、二元语法（bigram）和三元语法（trigram）。例如，长度为4的序列$w_1, w_2, w_3, w_4$在一元语法、二元语法和三元语法中的概率分别为
$$
\begin{aligned}
P(w_1, w_2, w_3, w_4) &=  P(w_1) P(w_2) P(w_3) P(w_4) ,\\
P(w_1, w_2, w_3, w_4) &=  P(w_1) P(w_2 \mid w_1) P(w_3 \mid w_2) P(w_4 \mid w_3) ,\\
P(w_1, w_2, w_3, w_4) &=  P(w_1) P(w_2 \mid w_1) P(w_3 \mid w_1, w_2) P(w_4 \mid w_2, w_3) .
\end{aligned}
$$
当$n$较小时，$n$元语法往往并不准确。例如，在一元语法中，由三个词组成的句子“你走先”和“你先走”的概率是一样的。然而，当$n$较大时，$n$元语法需要计算并存储大量的词频和多词相邻频率。

一元语法： 不准确，错误率高

多元语法： 计算，存储量大，计算各种情况的条件概率，指数爆炸。

那么，有没有方法在语言模型中更好地平衡以上这两点呢？我们将在本章探究这样的方法。

## 小结

- 语言模型是自然语言处理的重要技术。
- $N$元语法是基于$n-1$阶马尔可夫链的概率语言模型，其中$n$权衡了计算复杂度和模型准确性。

# 循环神经网络 RNN Recurrent Neural Network

循环神经网络。它并非刚性地记忆所有固定长度的序列，例如记住所有状态下的条件概率。

而是通过**隐藏状态**来存储之前时间步的信息。首先我们回忆一下前面介绍过的多层感知机，然后描述如何添加隐藏状态来将它变成循环神经网络。

## 含隐藏状态的循环神经网络

现在我们考虑输入数据存在时间相关性的情况。



假设$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$是序列中时间步$t$的小批量输入，$\boldsymbol{H}_t  \in \mathbb{R}^{n \times h}$是该时间步的隐藏变量。

> 比如输入了 $t$个小batch, 某一时间输入的是$\boldsymbol{X}_t$ ，此时对应的隐含层输出是$\boldsymbol{H}_t $ ,那如果不是第一个输入，应该还有前一个时刻的隐含层输出$\boldsymbol{H}_{t-1} $ 。

与多层感知机不同的是，这里我们保存上一时间步的隐藏变量$\boldsymbol{H}_{t-1}$，并引入一个新的权重参数$\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$，该参数用来描述在当前时间步如何使用上一时间步的隐藏变量。具体来说，时间步$t$的隐藏变量的计算由当前时间步的输入和上一时间步的隐藏变量共同决定：

$$\boldsymbol{H}_t = \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hh}  + \boldsymbol{b}_h).$$

与多层感知机相比，我们在这里添加了$\boldsymbol{H}_{t-1} \boldsymbol{W}_{hh}$一项。由上式中相邻时间步的隐藏变量$\boldsymbol{H}_t$和$\boldsymbol{H}_{t-1}$之间的关系可知，这里的隐藏变量能够捕捉截至当前时间步的序列的历史信息，就像是神经网络当前时间步的状态或记忆一样。因此，该隐藏变量也称为隐藏状态。由于隐藏状态在当前时间步的定义**使用了上一时间步的隐藏状态**，**上式的计算是循环的**。使用循环计算的网络即循环神经网络（recurrent neural network）。

循环神经网络有很多种不同的构造方法。含上式所定义的隐藏状态的循环神经网络是极为常见的一种。若无特别说明，本章中的循环神经网络均基于上式中隐藏状态的循环计算。在时间步$t$，输出层的输出和多层感知机中的计算类似：

$$\boldsymbol{O}_t = \boldsymbol{H}_t \boldsymbol{W}_{hq} + \boldsymbol{b}_q.$$

循环神经网络的参数包括隐藏层的权重$\boldsymbol{W}_{xh} \in \mathbb{R}^{d \times h}$、$\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$和偏差 $\boldsymbol{b}_h \in \mathbb{R}^{1 \times h}$，以及输出层的权重$\boldsymbol{W}_{hq} \in \mathbb{R}^{h \times q}$和偏差$\boldsymbol{b}_q \in \mathbb{R}^{1 \times q}$。值得一提的是，即便在不同时间步，循环神经网络也始终使用这些模型参数。因此，循环神经网络模型参数的数量不随时间步的增加而增长。

图6.1展示了循环神经网络在3个相邻时间步的计算逻辑。在时间步$t$，隐藏状态的计算可以看成是将输入$\boldsymbol{X}_t$和前一时间步隐藏状态$\boldsymbol{H}_{t-1}$连结后输入一个激活函数为$\phi$的全连接层。该全连接层的输出就是当前时间步的隐藏状态$\boldsymbol{H}_t$，且模型参数为$\boldsymbol{W}_{xh}$与$\boldsymbol{W}_{hh}$的连结，偏差为$\boldsymbol{b}_h$。当前时间步$t$的隐藏状态$\boldsymbol{H}_t$将参与下一个时间步$t+1$的隐藏状态$\boldsymbol{H}_{t+1}$的计算，并输入到当前时间步的全连接输出层。

<div align=center>
<img width="500" src="../imgs/rnn.png"/>
</div>
<div align=center>图6.1 含隐藏状态的循环神经网络</div>



我们刚刚提到，隐藏状态中$\boldsymbol{X}_t \boldsymbol{W}_{xh} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hh}$的计算等价于$\boldsymbol{X}_t$与$\boldsymbol{H}_{t-1}$连结后的矩阵乘以$\boldsymbol{W}_{xh}$与$\boldsymbol{W}_{hh}$连结后的矩阵。接下来，我们用一个具体的例子来验证这一点。首先，我们构造矩阵`X`、`W_xh`、`H`和`W_hh`，它们的形状分别为(3, 1)、(1, 4)、(3, 4)和(4, 4)。将`X`与`W_xh`、`H`与`W_hh`分别相乘，再把两个乘法运算的结果相加，得到形状为(3, 4)的矩阵。

```python
import torch

X, W_xh = torch.randn(3, 1), torch.randn(1, 4)
H, W_hh = torch.randn(3, 4), torch.randn(4, 4)
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)
```

输出：

```
tensor([[ 5.2633, -3.2288,  0.6037, -1.3321],
        [ 9.4012, -6.7830,  1.0630, -0.1809],
        [ 7.0355, -2.2361,  0.7469, -3.4667]])
```

将矩阵`X`和`H`按列（维度1）连结，连结后的矩阵形状为(3, 5)。可见，连结后矩阵在维度1的长度为矩阵`X`和`H`在维度1的长度之和（$1+4$）。然后，将矩阵`W_xh`和`W_hh`按行（维度0）连结，连结后的矩阵形状为(5, 4)。最后将两个连结后的矩阵相乘，得到与上面代码输出相同的形状为(3, 4)的矩阵。

```python
torch.matmul(torch.cat((X, H), dim=1), torch.cat((W_xh, W_hh), dim=0))
```

输出：

```
tensor([[ 5.2633, -3.2288,  0.6037, -1.3321],
        [ 9.4012, -6.7830,  1.0630, -0.1809],
        [ 7.0355, -2.2361,  0.7469, -3.4667]])
```

这里说的是一个RNN计算实现上的一个tricks， 

`torch.matmul(X, W_xh) + torch.matmul(H, W_hh) =  torch.matmul(torch.cat((X, H), dim=1), torch.cat((W_xh, W_hh), dim=0))`

## 应用：基于字符级循环神经网络的语言模型

最后我们介绍如何应用循环神经网络来构建一个语言模型。设小批量中样本数为1，文本序列为“想”“要”“有”“直”“升”“机”。图6.2演示了如何使用循环神经网络基于当前和过去的字符来预测下一个字符。在训练时，我们对每个时间步的输出层输出使用softmax运算，然后使用交叉熵损失函数来计算它与标签的误差。在图6.2中，由于隐藏层中隐藏状态的循环计算，时间步3的输出$\boldsymbol{O}_3$取决于文本序列“想”“要”“有”。 由于训练数据中该序列的下一个词为“直”，时间步3的损失将取决于该时间步基于序列“想”“要”“有”生成下一个词的概率分布与该时间步的标签“直”。

<div align=center>
<img width="500" src="../imgs/rnn_train.png"/>
</div>
<div align=center>图6.2 基于字符级循环神经网络的语言模型。</div>

因为每个输入词是一个字符，因此这个模型被称为字符级循环神经网络（character-level recurrent neural network）。因为不同字符的个数远小于不同词的个数（对于英文尤其如此），所以字符级循环神经网络的计算通常更加简单。在接下来的几节里，我们将介绍它的具体实现。

## 小结

- 使用循环计算的网络即循环神经网络。
- 循环神经网络的隐藏状态可以捕捉截至当前时间步的序列的历史信息。
- 循环神经网络模型参数的数量不随时间步的增加而增长。
- 可以基于字符级循环神经网络来创建语言模型。

# 语言模型数据集（周杰伦专辑歌词）

本节将介绍如何预处理一个语言模型数据集，并将其转换成字符级循环神经网络所需要的输入格式。为此，我们收集了周杰伦从第一张专辑《Jay》到第十张专辑《跨时代》中的歌词，并在后面几节里应用循环神经网络来训练一个语言模型。当模型训练好后，我们就可以用这个模型来创作歌词。

## 6.3.1 读取数据集

首先读取这个数据集，看看前40个字符是什么样的。




In [54]:
import torch
import random
import zipfile
import sys
sys.path.append("..") 

with zipfile.ZipFile('../Data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
corpus_chars[:40]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

这个数据集有6万多个字符。为了打印方便，我们把换行符替换成空格，然后仅使用前1万个字符来训练模型。

In [55]:
len(corpus_chars)
corpus_chars = corpus_chars.replace('\n',' ').replace('\r', ' ')
corpus_chars = corpus_chars[:10000]
len(corpus_chars)

10000

## 建立字库 字符索引
### 去重，把歌词里不重复的字取出来，建立词库，然后建立歌词对应于词库的索引表

In [56]:
vob_set = set(corpus_chars)
char_to_idx = dict([(char, i ) for i, char in enumerate(list(vob_set)) ])
print(len(char_to_idx))
idx_to_char = list(vob_set)

1027


In [57]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [651, 891, 377, 997, 355, 1019, 128, 651, 891, 871, 1016, 533, 201, 751, 220, 171, 128, 651, 891, 871]


In [58]:
char_to_idx['想']

651

In [59]:
idx_to_char[598]

'乌'

In [87]:
def load_data_jay_lyrics():
    """加载周杰伦歌词数据集"""
    with zipfile.ZipFile('../Data/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

## 时序数据的采样

在训练中我们需要每次随机读取小批量样本和标签。与之前章节的实验数据不同的是，时序数据的一个样本通常包含连续的字符。假设时间步数为5，样本序列为5个字符，即“想”“要”“有”“直”“升”。该样本的标签序列为这些字符分别在训练集中的下一个字符，即“要”“有”“直”“升”“机”。我们有两种方式对时序数据进行采样，分别是随机采样和相邻采样。

###  随机采样

下面的代码每次从数据里随机采样一个小批量。其中批量大小`batch_size`指每个小批量的样本数，`num_steps`为每个样本所包含的时间步数。
在随机采样中，每个样本是原始序列上任意截取的一段序列。**相邻的两个随机小批量**在**原始序列上**的位置*不*一定**相毗邻**。因此，我们无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态。在训练模型时，**每次随机采样前都需要重新初始化隐藏状态**。

In [61]:
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为输出的索引x是相应输入的索引y加1
    num_examples = (len(corpus_indices) - 1) // num_steps
    epoch_size = num_examples // batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)

    # 返回从pos开始的长为num_steps的序列
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(epoch_size):
        # 每次读取batch_size个随机样本
        i = i * batch_size
        batch_indices = example_indices[i: i + batch_size]
        X = [_data(j * num_steps) for j in batch_indices]
        Y = [_data(j * num_steps + 1) for j in batch_indices]
        yield torch.tensor(X, dtype=torch.float32, device=device), torch.tensor(Y, dtype=torch.float32, device=device)
        

In [135]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [12., 13., 14., 15., 16., 17.]], device='cuda:0') 
Y: tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [13., 14., 15., 16., 17., 18.]], device='cuda:0') 

X:  tensor([[18., 19., 20., 21., 22., 23.],
        [ 6.,  7.,  8.,  9., 10., 11.]], device='cuda:0') 
Y: tensor([[19., 20., 21., 22., 23., 24.],
        [ 7.,  8.,  9., 10., 11., 12.]], device='cuda:0') 



###  相邻采样

除对原始序列做随机采样之外，我们还可以**令相邻的两个随机小批量**在**原始序列上的位置相毗邻**。

这时候，我们就可以用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态，
从而使下一个小批量的输出也取决于当前小批量的输入，并如此循环下去。这对实现循环神经网络造成了两方面影响：

一方面，在训练模型时，我们只需在**每一个迭代周期开始时初始化隐藏状态**；

另一方面，当多个相邻小批量通过传递隐藏状态串联起来时，**模型参数的梯度计算将依赖所有串联起来的小批量序列**。
同一迭代周期中，随着迭代次数的增加，**梯度的计算开销会越来越大。**
为了使模型参数的梯度计算只依赖一次迭代读取的小批量序列，我们可以在每次读取小批量前将隐藏状态从计算图中分离出来。我们将在下一节（循环神经网络的从零开始实现）的实现中了解这种处理方式。

In [133]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices = torch.tensor(corpus_indices, dtype=torch.float32, device=device)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0: batch_size*batch_len].view(batch_size, batch_len)
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [83]:
my_seq = list(range(40))

# corpus_indices, char_to_idx, idx_to_char, vocab_size = load_data_jay_lyrics()    

# for X, Y in data_iter_consecutive(corpus_indices, 12, 10, device):
#     print(X.shape)

for X, Y in data_iter_consecutive(my_seq, batch_size=3, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
        [13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.],
        [26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.]],
       device='cuda:0')
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [13., 14., 15., 16., 17., 18.],
        [26., 27., 28., 29., 30., 31.]], device='cuda:0')
X:  tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [13., 14., 15., 16., 17., 18.],
        [26., 27., 28., 29., 30., 31.]], device='cuda:0') 
Y: tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [14., 15., 16., 17., 18., 19.],
        [27., 28., 29., 30., 31., 32.]], device='cuda:0') 

tensor([[ 6.,  7.,  8.,  9., 10., 11.],
        [19., 20., 21., 22., 23., 24.],
        [32., 33., 34., 35., 36., 37.]], device='cuda:0')
X:  tensor([[ 6.,  7.,  8.,  9., 10., 11.],
        [19., 20., 21., 22., 23., 24.],
        [32., 33., 34., 35., 36., 37.]], device='cuda:0') 
Y: tensor([[ 7.,  8.,  9., 10., 11., 12.],
  

# 循环神经网络的从零开始实现

在本节中，我们将从零开始实现一个基于字符级循环神经网络的语言模型，并在周杰伦专辑歌词数据集上训练一个模型来进行歌词创作。首先，我们读取周杰伦专辑歌词数据集：

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()    

## one-hot向量

为了将词表示成向量输入到神经网络，一个简单的办法是使用one-hot向量。假设词典中不同字符的数量为$N$（即词典大小`vocab_size`），每个字符已经同一个从0到$N-1$的连续整数值索引一一对应。如果一个字符的索引是整数$i$, 那么我们创建一个全0的长为$N$的向量，并将其位置为$i$的元素设成1。该向量就是对原字符的one-hot向量。下面分别展示了索引为0和2的one-hot向量，向量长度等于词典大小。

## RNN的输入

### 对于文本信息来说，一般文本中的“词” word 被构造为词向量，而这里汉字被构造为 One-hot向量。

用一个010000的独热编码来表示一个词库中的一个元素，来对文字信息进行编码。


In [24]:
import torch.nn.functional as F
import numpy as np

X = torch.arange(10).view(2, 5)
onet_hot_features = F.one_hot(X, torch.tensor(10))

In [25]:
onet_hot_features.shape

torch.Size([2, 5, 10])

我们每次采样的小批量的形状是(批量大小, 时间步数)。下面的函数将这样的小批量变换成数个可以输入进网络的形状为(批量大小, 词典大小)的矩阵，矩阵个数等于时间步数。也就是说，时间步$t$的输入为$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$，其中$n$为批量大小，$d$为输入个数，即one-hot向量长度（词典大小）。

In [217]:
def one_hot(x, n_class, dtype=torch.float32): 
    # X shape: (batch), output shape: (batch, n_class)
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res

# 本函数已保存在d2lzh_pytorch包中方便以后使用
def to_onehot(X, n_class):  
    # X shape: (batch, seq_len), output: seq_len elements of (batch, n_class)
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]


import torch.nn.functional as F

def to_one_hot(X, n_class):
    assert isinstance(X, torch.Tensor)
    X.to('cuda' if torch.cuda.is_available() else 'cpu')
    #print(X[:, 0].long())
    return [F.one_hot(X[:, i].long(), n_class ).float() for i in range(X.shape[1])]
    
def mydata_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices = torch.tensor(corpus_indices, dtype=torch.float32, device=device)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0: batch_size*batch_len].view(batch_size, batch_len)
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y
        


#X = torch.arange(10).view(2, 5)
class_num  = 10
my_seq = torch.tensor(list(range(class_num)))
for X, Y in mydata_iter_consecutive(my_seq, batch_size=2, num_steps=3):
    print('X: ', X, '\nY:', Y, '\n')
    
    one_hot_inputs = to_one_hot(X, class_num)
    
    inputs = to_onehot(X, class_num)

    print(len(inputs), inputs , inputs[0].shape)
    print(len(one_hot_inputs), one_hot_inputs , one_hot_inputs[0].shape)
    
    for i in range(len(inputs)):
        print(torch.equal(inputs[i], one_hot_inputs[i]))
        

d:\python36\lib\site-packages\ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


X:  tensor([[0., 1., 2.],
        [5., 6., 7.]], device='cuda:0') 
Y: tensor([[1., 2., 3.],
        [6., 7., 8.]], device='cuda:0') 

3 [tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]], device='cuda:0'), tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], device='cuda:0'), tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], device='cuda:0')] torch.Size([2, 10])
3 [tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]], device='cuda:0'), tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], device='cuda:0'), tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], device='cuda:0')] torch.Size([2, 10])
True
True
True


In [147]:
out = (torch.randn(3,5),torch.randn(3,5))
out = torch.cat(out, dim = 0)
print(out.shape)

torch.Size([6, 5])


## 初始化模型参数

接下来，我们初始化模型参数。隐藏单元个数 `num_hiddens`是一个超参数。

In [205]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)

    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])

will use cuda


## 定义模型

我们根据循环神经网络的计算表达式实现该模型。首先定义`init_rnn_state`函数来返回初始化的隐藏状态。它返回由一个形状为(批量大小, 隐藏单元个数)的值为0的`NDArray`组成的元组。使用元组是为了更便于处理隐藏状态含有多个`NDArray`的情况。

In [206]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

下面的`rnn`函数定义了在一个时间步里如何计算隐藏状态和输出。这里的激活函数使用了tanh函数。3.8节（多层感知机）中介绍过，当元素在实数域上均匀分布时，tanh函数值的均值为0。

In [207]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []

    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
        
    return outputs, (H,)

# 怎么理解RNN的输入 

## Num_step

一段序列中，具有多少个词，例如将num_steps设置为4，则采样的是以4个词作为步长，
例如“是不是傻”， 这个可以看作是一条数据。

## one_hot_size
“是不是傻” 这是文字，需要用词库编码为one-hot向量，或者其他词向量等形式，
那样一个词就被映射为[vocab_size]大小的向量。

对于一个长度为： num_step的数据， 一条数据大小就有[num_step, vocab_size]
## batch_size
只用一条数据训练显然是不行的，所以要引入一个batch,而这里的batch如何理解，例如batch_size为3：

“是不是傻啊”

“你是傻是吗” 



这样构成的数据大小为**\[batch_size, num_step, vocab_size\] = 【2条数据， 每条5个词， 每个词one-hot 1027维】**

但是我们训练的时候都是二维的矩阵相乘，这里回到RNN的原始概念，利用上一层隐含层的输出，结合当前状态的输入，来预测下一个：“可能的数据”

所以输入的数据应该是 **\[batch_size, vocab_size\]=【2条数据， 1027维度】**，5个词 num_steps维度作为隐状态迭代次数。

例如：

将“是”“你”  的独热向量作为输入，而其输出, 而num_steps 有5个，所以会通过这进行num_steps次状态迭代，得到最终预测的一组输出，而隐含层输出也可以记录下来，所以会有5 条**\[batch_size , vocab_size\]** 大小的数据，每个数据是预测的下一个词的独热编码。




In [218]:
# import torch.nn as nn
# def to_onehot(X, vocab_size):
#     return F.one_hot((X.view(X.shape[1],-1)).to(device), vocab_size)

state = init_rnn_state(X.shape[0], num_hiddens, device)

#inputs = to_onehot(X.to(device), 10)
X = torch.arange(10).view(2, 5).to('cuda')

#onet_hot_features = F.one_hot(X.to(device), vocab_size)

inputs = to_one_hot(X,vocab_size)
print(inputs)
params = get_params()


outputs, state_new = rnn([data.float() for data in inputs], state, params)

print(len(outputs), outputs[0].shape, state_new[0].shape) 

[tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
5 torch.Size([2, 1027]) torch.Size([2, 256])


##  定义预测函数

以下函数基于前缀`prefix`（含有数个字符的字符串）来预测接下来的`num_chars`个字符。这个函数稍显复杂，其中我们将循环神经单元`rnn`设置成了函数参数，这样在后面小节介绍其他循环神经网络时能重复使用这个函数。

In [224]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    #print(output)
    for t in range(num_chars + len(prefix) - 1):
        #print('pre fix {}'.format(t))
        # 将上一时间步的输出作为当前时间步的输入
        #print('input is {}'.format(idx_to_char[output[-1]]))
        X = to_one_hot(torch.tensor([[output[-1]]], device=device), vocab_size)
        #print(X)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
            
    return ''.join([idx_to_char[i] for i in output])

我们先测试一下`predict_rnn`函数。我们将根据前缀“分开”创作长度为10个字符（不考虑前缀长度）的一段歌词。因为模型参数为随机值，所以预测结果也是随机的。

In [225]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)

[948]
pre fix 0
input is 分
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 1
input is 开
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 2
input is 便
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 3
input is 朽
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 4
input is 蝙
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 5
input is 险
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 6
input is 钩
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 7
input is 题
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 8
input is 啊
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 9
input is 啸
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 10
input is 杵
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]


'分开便朽蝙险钩题啊啸杵山'

## 裁剪梯度

循环神经网络中较容易出现梯度衰减或梯度爆炸。我们会在6.6节（通过时间反向传播）中解释原因。为了应对梯度爆炸，我们可以裁剪梯度（clip gradient）。假设我们把所有模型参数梯度的元素拼接成一个向量 $\boldsymbol{g}$，并设裁剪的阈值是$\theta$。裁剪后的梯度

$$ \min\left(\frac{\theta}{\|\boldsymbol{g}\|}, 1\right)\boldsymbol{g}$$

的$L_2$范数不超过$\theta$。

In [69]:
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

##  困惑度

我们通常使用困惑度（perplexity）来评价语言模型的好坏。回忆一下3.4节（softmax回归）中交叉熵损失函数的定义。困惑度是对交叉熵损失函数做指数运算后得到的值。特别地，

- 最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
- 最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
- 基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。

显然，任何一个有效模型的困惑度必须小于类别个数。在本例中，困惑度必须小于词典大小`vocab_size`。

## 定义模型训练函数

跟之前章节的模型训练函数相比，这里的模型训练函数有以下几点不同：

1. 使用困惑度评价模型。
2. 在迭代模型参数前裁剪梯度。
3. 对时序数据采用不同采样方法将导致隐藏状态初始化的不同。相关讨论可参考6.3节（语言模型数据集（周杰伦专辑歌词））。

另外，考虑到后面将介绍的其他循环神经网络，为了更通用，这里的函数实现更长一些。

In [226]:
import time
import dl_utils
import math
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  
            # 否则需要使用detach函数从计算图分离隐藏状态, 这是为了
            # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                for s in state:
                    s.detach_()
            #X = torch.Tensor(X)
            #print(x.device)
            inputs = to_one_hot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            
            outputs = torch.cat(outputs, dim=0)
            
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())
            
            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            dl_utils.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

## 训练模型并创作歌词

现在我们可以训练模型了。首先，设置模型超参数。我们将根据前缀“分开”和“不分开”分别创作长度为50个字符（不考虑前缀长度）的一段歌词。我们每过50个迭代周期便根据当前训练的模型创作一段歌词。

In [227]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [228]:
import torch.nn as nn
def one_hot(x, n_class, dtype=torch.float32): 
    # X shape: (batch), output shape: (batch, n_class)
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res

# 本函数已保存在d2lzh_pytorch包中方便以后使用
def to_onehot(X, n_class):  
    # X shape: (batch, seq_len), output: seq_len elements of (batch, n_class)
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 70.646543, time 0.48 sec
[948]
pre fix 0
input is 分
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 1
input is 开
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 2
input is  
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 3
input is 我
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 4
input is 不
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 5
input is 要
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 6
input is 再
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 7
input is 想
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 8
input is  
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 9
input is 我
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 10
input is 不
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 11
input is 要
[tensor([[0., 0., 0.,  ..., 0., 0.

[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 43
input is  
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 44
input is 我
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 45
input is 不
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 46
input is  
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 47
input is 我
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 48
input is 不
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 49
input is  
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 50
input is 我
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 51
input is 不
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
 - 不分开堡 我不要再想 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 
epoch 150, perplexity 2.962483, time 0.48 sec
[948]
pre fix 0
input is 分
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 1
in

[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 33
input is 瞎
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 34
input is  
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 35
input is 说
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 36
input is 底
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 37
input is 了
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 38
input is 打
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 39
input is 阳
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 40
input is 光
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 41
input is  
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 42
input is 一
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 43
input is 言
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
pre fix 44
input is 主
[tensor([[0., 0.

In [134]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 64.308868, time 0.35 sec
 - 分开 我想要这想 我不要再不 我不要再想 我不要再不 我不要再想 我不要再不 我不要再想 我不要再不 我
 - 不分开 我想要这 我有了双截 谁不的美 在子我的见 我不要你 你有我有 我想我这多 是不的没不 我不要再想
epoch 100, perplexity 7.124937, time 0.35 sec
 - 分开 我不要这样瘦着你的姑不 我的星写 你来了一凉 银制茶烛 温暖了外屋 白制蜡烛 温暖了空屋 白色蜡烛
 - 不分开觉 你样经离开我 后知不觉 我该了这节活 后知后觉 我该了这节活 后知后觉 我该了好生活 后知后觉 
epoch 150, perplexity 2.044541, time 0.34 sec
 - 分开 我不要再想瘦 你爱你的天 我在魂球 在水怕空的 白有蜡在 全我了空的溪边河口默默等著我 娘子依旧每
 - 不分开觉你 我以多你想离 如果我遇见你是一场悲剧 我想我这辈子注定一个人演戏 最后再一个人慢慢的回忆 没有
epoch 200, perplexity 1.303947, time 0.35 sec
 - 分开 她候我 谁不让没我手 你说啊 是不是你不想活 说你怎么面对我 甩开球我满腔的怒火 我想揍你已经很久
 - 不分开觉 你样经离 你去再蒙 怎么作痛 你都没痛 说我不懂 说了没用 他的笑容 有何不同 在你心中 我不再
epoch 250, perplexity 1.165118, time 0.34 sec
 - 分开 她候的 旧时么 装满了明信代的铁盒里藏著一片玫瑰花瓣 黄金葛爬满了雕花的门窗 夕阳斜斜映在斑驳的砖
 - 不分开觉你 我爱要你想经 不果我遇见 一什是我感无 它下妙传出手 漂亮的假动作 帅呆了我 说你说 分数怎么


## Pytorch 循环神经网络的简洁实现

In [148]:

num_hiddens = 256
# rnn_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens) # 已测试
rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens)

num_steps = 35
batch_size = 2
state = None
X = torch.rand(num_steps, batch_size, vocab_size)
Y, state_new = rnn_layer(X, state)
print(Y.shape, len(state_new), state_new[0].shape)

torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [152]:
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1) 
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)
        self.state = None

    def forward(self, inputs, state): # inputs: (batch, seq_len)
        # 获取one-hot向量表示
        X = to_onehot(inputs, vocab_size) # X是个list
        Y, self.state = self.rnn(torch.stack(X), state)
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output = self.dense(Y.view(-1, Y.shape[-1]))
        return output, self.state

In [153]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]] # output会记录prefix加上输出
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        if state is not None:
            if isinstance(state, tuple): # LSTM, state:(h, c)  
                state = (state[0].to(device), state[1].to(device))
            else:   
                state = state.to(device)
            
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [154]:

model = RNNModel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('分开', 10, model, vocab_size, device, idx_to_char, char_to_idx)

'分开箭太泽跑泽泽泽泽泽泽'

In [159]:
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    state = None
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态, 这是为了
                # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state = (state[0].detach(), state[1].detach())
                else:   
                    state = state.detach()
    
            (output, state) = model(X, state) # output: 形状为(num_steps * batch_size, vocab_size)
            
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            # 梯度裁剪
            grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        
        try:
            perplexity = math.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

In [160]:

num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 10.323663, time 0.09 sec
 - 分开始不起 爱你在我 你的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的
 - 不分开  一场悲剧 我想我这样你的手 一不的口  我不要 你想我这样你的人  不知道  我不了 不你我想 
epoch 100, perplexity 1.278276, time 0.09 sec
 - 分开 一起心动的黑色幽默 你的完美主义 太彻底 让我连恨都难以下笔 将真心抽离写成日记 像是一场默剧 你
 - 不分开始 我不懂 爱你在练 我有你爸 你笑手 一定 我感到的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我
epoch 150, perplexity 1.063667, time 0.10 sec
 - 分开 一起心中 在 已经过人雨 风沉默没开  我上没有你 相思 说 泪怎么空 我都说的 从小到真的在我吸
 - 不分开不了我不能再想 我不 再不 我不能 爱情走的太快就像龙卷风 不能承受我已无处可躲 我不要再想 我不要
epoch 200, perplexity 1.031684, time 0.10 sec
 - 分开 一起心中 在 有神个人让后知后觉 我该好好生活 我该好好生活 不知不觉 你已经离开我 不知不觉 我
 - 不分开不了我不能 说你怎么面对我 甩开球 满腔的怒火 我想揍你已经很久 别想躲 说你眼睛看着我 别发抖 快
epoch 250, perplexity 1.058628, time 0.11 sec
 - 分开 一起心中 夕阳有钟映在斑驳的砖墙 铺著榉木板的屋内还弥漫 姥姥当年酿的豆瓣酱 我对著黑白照片开始想
 - 不分开不了我不能 平常话不多 除非是乌鸦抢了它的窝 它在灌木丛旁邂逅 一只令它心仪的母斑鸠 印地安老斑鸠 


# GRU

In [161]:


num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
    
    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])


def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + R * torch.matmul(H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)


num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']


train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

will use cuda
epoch 40, perplexity 153.093699, time 0.94 sec
 - 分开 我想你的让我不想想想想你你的可爱女人 坏坏的让我不想想想想你你的可爱女人 坏坏的让我不想想想想你你
 - 不分开 你不的让我不想想想想你你的可爱女人 坏坏的让我不想想想想你你的可爱女人 坏坏的让我不想想想想你你的
epoch 80, perplexity 32.050503, time 0.95 sec
 - 分开 我想要这样 我不要再想 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我
 - 不分开 我不能再想 我不要再想 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我
epoch 120, perplexity 4.807658, time 0.96 sec
 - 分开 我想要这样坦堡 就这样的生活 我爱你 你爱我 我不能再想你 你爱你依不舍 不知不觉 我跟了这节奏 
 - 不分开 不要再这样打我妈妈 我说的话你甘会听 不要再这样打我妈妈 我说的话你甘会听 不要再这样打我妈妈 我
epoch 160, perplexity 1.488510, time 0.95 sec
 - 分开 我想要这样牵堡在这样 想要和你融化在一起 融化在宇宙里 我每天每天每天在想想想想著你 这样的甜蜜 
 - 不分开 不要再这样打我妈妈 难道你手不会痛吗 其实我回家就想要阻止一切 让家庭回到过去甜甜 温馨的欢乐香味


In [163]:
lr = 1e-2
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = RNNModel(gru_layer, vocab_size).to(device)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.024476, time 0.11 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻       所有回忆对着我进攻    
 - 不分开不要再这样打我妈妈 我说的话 你甘会听 不要再这样打我妈妈 难道你手不会痛吗 其实我回家就想要阻止一
epoch 80, perplexity 1.013490, time 0.10 sec
 - 分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
 - 不分开不要再这样打我妈妈 我说的话 你甘会听 不要再这样打我妈妈 难道你手不会痛吗 其实我回家就想要阻止一
epoch 120, perplexity 1.010614, time 0.11 sec
 - 分开 它在空中停留 所有人看着我 抛物线进球 单手过人运球 篮下妙传出手 漂亮的假动作 帅呆了我 全场盯
 - 不分开 从小到大只有妈妈的温暖  为什么我爸爸 那么凶 如果真的我有一双翅膀 二双翅膀 随时出发 偷偷出发
epoch 160, perplexity 1.010488, time 0.11 sec
 - 分开 它在空中停留 所有人看着我 抛物线进球 单手过人运球 篮下妙传出手 漂亮的假动作 帅呆了我 全场盯
 - 不分开 从小到大只有妈妈的温暖  为什么我爸爸 那么凶 如果真的我有一双翅膀 二双翅膀 随时出发 偷偷出发


# LSTM

In [164]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))
    
    W_xi, W_hi, b_i = _three()  # 输入门参数
    W_xf, W_hf, b_f = _three()  # 遗忘门参数
    W_xo, W_ho, b_o = _three()  # 输出门参数
    W_xc, W_hc, b_c = _three()  # 候选记忆细胞参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])

will use cuda


In [165]:
def init_lstm_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), 
            torch.zeros((batch_size, num_hiddens), device=device))

def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid(torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i)
        F = torch.sigmoid(torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f)
        O = torch.sigmoid(torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o)
        C_tilda = torch.tanh(torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

In [166]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [167]:
train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 210.208085, time 1.19 sec
 - 分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
 - 不分开 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我 我不的我
epoch 80, perplexity 64.678468, time 1.13 sec
 - 分开 我想你这你 我不要这样 我不要这生 我不要这生 我不要这生 我不要这生 我不要这生 我不要这生 我
 - 不分开 我想你这你 我不要这样 我不要这生 我不要这生 我不要这生 我不要这生 我不要这生 我不要这生 我
epoch 120, perplexity 15.337807, time 1.09 sec
 - 分开 我想我这你很着你 我想想你 你不要再看着 我想 你想很久久我 说散你的手笑有你 别么  别给我 说
 - 不分开 我想你你 你我的一话人 我想想你 我不要再难样 我想想你 你我的难手  没有你对我有多难难熬多多 
epoch 160, perplexity 4.113094, time 1.10 sec
 - 分开 我说 你爱我的吧? 我想想你已睡很  想样的生笑笑每的看不不  我知道这里很美但家乡的你更美 就血
 - 不分开 我已你你 我不多再熬奏 我不要这生活 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活


In [ ]:
lr = 1e-2 # 注意调整学习率
lstm_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(lstm_layer, vocab_size)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)